In [12]:
import pandas as pd
import os
import re
import tarfile

### File Prep - Skip if Already Run 

In [14]:
tar_path = './data/UNGDC_1946-2023.tgz'
extract_path = './data/UNGDC_1946-2023/'

with tarfile.open(tar_path, 'r:gz') as tar:
    tar.extractall(path=extract_path)

### Data Import and Notebook Setup

In [ ]:
vdem_df = pd.read_csv('./data/V-Dem-CY-Full+Others-v14.csv')

C:\Users\maxla\AppData\Local\Temp\ipykernel_48352\952016332.py:1: DtypeWarning: Columns (364,365,366,399,415,804,836,837,924,1240,1257,1486,3094,3168,3169,3341,3342,3344,3345,3347,3350,3352) have mixed types. Specify dtype option on import or set low_memory=False.
  vdem_df = pd.read_csv('./data/V-Dem-CY-Full+Others-v14.csv')


In [7]:
print(vdem_df.head())
print(vdem_df.info())

  country_name country_text_id  country_id  year historical_date  project  \
0       Mexico             MEX           3  1789      1789-12-31        1   
1       Mexico             MEX           3  1790      1790-12-31        1   
2       Mexico             MEX           3  1791      1791-12-31        1   
3       Mexico             MEX           3  1792      1792-12-31        1   
4       Mexico             MEX           3  1793      1793-12-31        1   

   historical                  histname  codingstart  codingend  ...  \
0           1  Viceroyalty of New Spain         1789       2023  ...   
1           1  Viceroyalty of New Spain         1789       2023  ...   
2           1  Viceroyalty of New Spain         1789       2023  ...   
3           1  Viceroyalty of New Spain         1789       2023  ...   
4           1  Viceroyalty of New Spain         1789       2023  ...   

   e_mipopula  e_miurbani  e_pefeliex  e_wb_pop  e_pechmor  e_miinteco  \
0         NaN         NaN     

In [ ]:
base_dir = './data/UNGDC_1946-2023/TXT'

speeches = []

# Walk through all subdirectories
for root, dirs, files in os.walk(base_dir):
    for file in files:
        # Process only .txt files
        if file.endswith('.txt'):
            # The expected naming pattern is: COUNTRY_SESSION_YEAR.txt
            # e.g., USA_75_2020.txt
            match = re.match(r"([A-Z]{3})_(\d{1,3})_(\d{4})\.txt", file)
            if match:
                country, session, year = match.groups()
            else:
                # If the file name doesn't match the expected pattern, skip it.
                continue

            # Build the full file path and read the content.
            file_path = os.path.join(root, file)
            try:
                with open(file_path, 'r', encoding='utf-8') as f:
                    text = f.read()
            except UnicodeDecodeError as e:
                # If you run into decoding issues, you might use errors='replace' or try a different encoding.
                print(f"Decoding error in {file_path}: {e}")
                continue

            speeches.append({
                'Country': country,
                'Session': int(session),
                'Year': int(year),
                'Text': text,
            })

# Create a DataFrame from the list
df_speeches = pd.DataFrame(speeches)
print(df_speeches.head())

  Country  Session  Year                                               Text  \
0     ARG        1  1946  At the resumption of the first session of the ...   
1     AUS        1  1946  The General Assembly of the United Nations is ...   
2     BEL        1  1946  The\tprincipal organs of the United Nations ha...   
3     BLR        1  1946  As more than a year has elapsed since the Unit...   
4     BOL        1  1946  Coming to this platform where so many distingu...   

                                            FilePath  
0  ./data/UNGDC_1946-2023/TXT\Session 01 - 1946\A...  
1  ./data/UNGDC_1946-2023/TXT\Session 01 - 1946\A...  
2  ./data/UNGDC_1946-2023/TXT\Session 01 - 1946\B...  
3  ./data/UNGDC_1946-2023/TXT\Session 01 - 1946\B...  
4  ./data/UNGDC_1946-2023/TXT\Session 01 - 1946\B...  
